**Сформируем с помощью LLM-агентов следующий датасет: `Описание фильма -> его название`.**

**Это очень полезно, так как бывают ситуации, когда помнишь какие-то отдельные детали сюжета фильма или сюжет в общих чертах или, например, режиссёра фильма, но название вспомнить не получается. В таких ситуациях можно пойти искать название фильма по описанию в Интернете, но иногда поиск ни к чему не приводит. LLM в свою очередь хорошо могут обрабатывать различные (по степени детализации в том числе) тексты и выделять в них закономерности, способны к таким сложным операциям, как суммаризация текста, перифраз и т.п. Поэтому они могут решать задачу поиска фильма по свободному описанию его сюжета гораздо эффективнее, чем детерминированные алгоритмы поиска.**

**При этом скорее всего LLM не смогут решать такую задачу для произвольного фильма => можно попробовать дообучить их этому на хороших синтетических данных.**

**Для генерации синтетических описаний сюжета можно воспользоваться двумя LLM-агентами: один - с RAG-модулем, содержащим информацию про сюжеты фильмов, - будет доставать информацию про сюжеты фильмов, а затем суммаризовывать их различными способами; второй - с поиском по Интернету - будет доставать детали про фильмы, в частности, - год выпуска фильма (для простоты). Затем предполагается скриптом слить эти данные в единый текст.**

**Информацию по сюжетам фильмов и по деталям будем брать из Википедии (это своего рода тоже некоторый энциклопедический документ про тот или иной фильм).**

**Создадим двух LLM-агентов, использующих (хранящиеся локально) модели Gemma2 2B (Google) в качестве основы. Я выбрал именно эту LLM за основу, так как две такие модели можно поместить на GPU/CPU (многие другие - нет). Но они конечно не очень мощные, что жалко... (Квантизацию пробовал, не сработало нормально для эмбеддера => нет смысла использовать эмбеддинги, полученные с помощью Gemma2 2B, одновременно с Gemma2 9B (так как эмбеддинги для Gemma2 9B и для Gemma2 2B будут совсем разные из-за различий в весах моделей))**

**Инсталляция библиотек:**

**Проверим версию библиотеки `transformers`, она должна быть не ниже 4.41.0.**

In [1]:
!pip show transformers

Name: transformers
Version: 4.42.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: kaggle-environments


**Всё ок.**

**Установим остальные необходимые библиотеки:**

In [2]:
!pip install langchain sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 25.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 62.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency reso

In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.3 MB/s eta 0:00:0000:0100:01


In [4]:
!pip install langchain-huggingface

In [5]:
!pip install bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 8.6 MB/s eta 0:00:00:00:0100:01


In [6]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=89fd01c1e6bfb02aa00914768a44845e3e696e25b13207ccc276b5b0abfd2dfe
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


**Сделаем RAG-модуль.**

In [7]:
import bs4
from langchain import hub
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm

**Достанем данные. Сделаем это следующим образом: возьмём первые 102 фильма (не спрашивайте, почему 102 :) ) 250 лучших по версии сайта IMDb фильмов (это для примера, далее БД можно конечно расширять), для каждого из фильмов найдём страницу в Википедии про него и достанем из неё секцию про сюжет. Далее надо "нарезать" текст, преобразовать в эмбеддинги и составить из полученных эмбеддингов  БД.**

**Поиск 250 лучших по версии IMDb фильмов, из них оставляем первые 102.**

In [8]:
import requests

url = "https://www.imdb.com/chart/top/"
headers = {'User-Agent':'Mozilla/5.0'}
r = requests.get(url, headers=headers)

html = r.text

In [9]:
from bs4 import BeautifulSoup
import json

soup = BeautifulSoup(html, "html.parser")

data = json.loads(soup.find_all('script')[2].contents[0])["itemListElement"]

top250_movies = []

for item in data:
    top250_movies.append(item["item"]["name"].replace("&apos;", "\'"))

In [10]:
top102_movies = top250_movies[:103]

**Найдём в Википедии сюжеты фильмов из полученного списка.**

In [11]:
top102_movies_alt = []

for mov_name in top102_movies:
    top102_movies_alt += [mov_name + " (film)"] 

In [12]:
import wikipedia

plot_descs = dict()

for mov_it in tqdm(range(len(top102_movies))):
    try:
        if mov_it == 28 or mov_it == 42:
            raise Exception
        if mov_it == 34:
            mov_page = wikipedia.page('Gladiator (2000 film)', auto_suggest=False)
        elif mov_it == 83:
            mov_page = wikipedia.page('Your Name', auto_suggest=False)
        elif mov_it == 98:
            mov_page = wikipedia.page('Lawrence of Arabia (film)', auto_suggest=False)
        elif mov_it == 101:
            mov_page = wikipedia.page('Oppenheimer (film)', auto_suggest=False)
        else:
            mov_page = wikipedia.page(top102_movies[mov_it], auto_suggest=False)
    except Exception as e:
        try:
            mov_page = wikipedia.page(top102_movies_alt[mov_it], auto_suggest=False)
        except Exception as e:
            if mov_it == 31:
                mov_page = wikipedia.page('The Pianist (2002 film)', auto_suggest=False)
            elif mov_it == 33:
                mov_page = wikipedia.page('Psycho (1960 film)', auto_suggest=False)
            elif mov_it == 41:
                mov_page = wikipedia.page('Whiplash (2014 film)', auto_suggest=False)
            elif mov_it == 65:
                mov_page = wikipedia.page('Witness for the Prosecution (1957 film)', auto_suggest=False)
            elif mov_it == 71:
                mov_page = wikipedia.page('Coco (2017 film)', auto_suggest=False)
            elif mov_it == 76:
                mov_page = wikipedia.page('American Beauty (1999 film)', auto_suggest=False)    
            elif mov_it == 82:
                mov_page = wikipedia.page('Joker (2019 film)', auto_suggest=False)
    plot = mov_page.section("Plot")
    plot_descs[top102_movies[mov_it]] = plot

 17%|█▋        | 17/103 [00:08<00:41,  2.06it/s]/opt/conda/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/conda/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
100%|██████████| 103/103 [01:07<00:00,  1.52it/s]


**Анализ показал, что сюжеты некоторых фильмов распарсить не удалось => выбросим пустые строки/None-объекты/слишком короткие описания.**

In [13]:
plot_descs_clean = dict()

for mov, plot in plot_descs.items():
    if plot is not None and plot != '' and mov != "The Godfather Part II": # описание сюжета второй части "Крёстного отца" крайне короткое 
        plot_descs_clean[mov] = plot 

In [14]:
len(plot_descs_clean)

97

**Определим максимальную и минимальную длину описания в наборе данных:**

In [15]:
max_len = 0

for mov, plot in plot_descs_clean.items():
    curr_len = len(f"Movie name: {mov}\n\n\n\nIts plot: {plot}")
    if curr_len > max_len:
        max_len = curr_len

print("max length:", max_len)

max length: 5743


In [16]:
min_len = 10000

for mov, plot in plot_descs_clean.items():
    curr_len = len(f"Movie name: {mov}\n\n\n\nIts plot: {plot}")
    if curr_len < min_len:
        min_len = curr_len

print("min length:", min_len)

min length: 2261


**Полученные фрагменты текста (вместе с названиями фильмов) не очень большие (даже если считать по буквам) => не будем делить их на куски для RAG (тем более для нас деление текста на куски будет чем-то плохим, так как хотим доставать фрагменты сюжета в правильной последовательности, а это обеспечить будет сложно).**

**Формируем Db для RAG.**

In [17]:
import huggingface_hub

huggingface_hub.notebook_login()

**Скачаем токенайзер и эмбеддинг-модель (так как работаем с Gemma 2B, то и для Db стоит брать токенайзер и эмбеддинги этой модели).**

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "google/gemma-2-2b-it" 

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
#emb_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto") 

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

**Превратим тексты в документы (при этом отформатируем их так, как нам нужно, то есть поместим названия фильмов перед текстами).**

In [19]:
source_docs = [
    Document(
        page_content=f"Movie name: {mov}\n\n\n\nIts plot: {plot}", metadata={"source": mov}
    ) for mov, plot in plot_descs_clean.items()
]

**Посмотрим на длины текстов в токенах.**

In [20]:
lengths = [len(tokenizer.encode(doc.page_content)) for doc in tqdm(source_docs)]

print(lengths)

100%|██████████| 97/97 [00:00<00:00, 430.85it/s]

[862, 970, 779, 835, 1111, 1228, 920, 869, 1030, 461, 848, 789, 903, 749, 914, 900, 821, 902, 925, 749, 815, 1069, 932, 959, 867, 747, 801, 930, 1053, 1055, 796, 941, 847, 894, 1039, 826, 909, 934, 744, 875, 622, 940, 896, 858, 598, 569, 796, 726, 1119, 990, 879, 871, 976, 707, 925, 952, 967, 702, 791, 703, 666, 777, 825, 822, 899, 885, 834, 897, 1176, 699, 760, 921, 589, 926, 958, 898, 905, 904, 750, 752, 1215, 900, 1031, 1005, 726, 823, 845, 490, 865, 792, 610, 565, 965, 1138, 870, 834, 902]


In [21]:
print("Max Length (in tokens):", max(lengths))
print("Min Length (in tokens):", min(lengths))

Max Length (in tokens): 1228
Min Length (in tokens): 461


**С учётом максимальной длины текста в токенах выставим максимальный chunk_size, равный 1300 (токенов).**

**Порежем документы на куски (на самом деле необязательный шаг, но сделал, так как в принципе этот шаг надо делать), преобразуем их в эмбеддинги, сформируем FAISS-векторную базу данных.**

In [31]:
# import torch
# import gc

# del vectordb
# del embedding_model

# torch.cuda.empty_cache()
# gc.collect()

5180

In [22]:
embedding_model = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

2024-08-15 23:37:37.898399: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 23:37:37.898526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 23:37:38.026451: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
from langchain_community.vectorstores.utils import DistanceStrategy

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=1300, chunk_overlap=0, strip_whitespace=True)
docs_processed = text_splitter.split_documents(source_docs)

vectordb = FAISS.from_documents(
     documents=docs_processed,
     embedding=embedding_model,
     distance_strategy=DistanceStrategy.COSINE
)

2024-08-15 23:38:33.119607: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 23:38:33.119662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 23:38:33.121090: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [24]:
for query in plot_descs_clean.keys():
    print(query, vectordb.similarity_search(query, k=1))

2024-08-15 23:40:22.634629: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 23:40:22.634696: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 23:40:22.636130: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


The Shawshank Redemption [Document(metadata={'source': 'The Departed'}, page_content="Movie name: The Departed\n\n\n\nIts plot: In the 1980s in Boston, Irish Mob boss Frank Costello introduces himself to a young Colin Sullivan.\nMany years later, Sullivan has been groomed as a spy inside the Massachusetts State Police (MSP) and joins the Special Investigation Unit (SIU). Another police academy recruit, Billy Costigan, is selected by Captain Queenan and Sergeant Dignam to go undercover as a criminal.\nCostigan serves a term in prison to set up his cover and further commits a series of crimes. Sullivan begins dating Madolyn Madden, a psychiatrist. Costigan catches the attention of Costello, who later recruits Costigan into his organization. Over the next year, Costigan becomes increasingly involved in the organization. His mental state declines but Queenan and Dignam convince him to continue. Costigan begins having appointments with Madolyn.\nThe MSP and Costello realize there is a mole 

2024-08-15 23:40:35.249825: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 23:40:35.249886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 23:40:35.251246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


The Godfather [Document(metadata={'source': 'The Departed'}, page_content="Movie name: The Departed\n\n\n\nIts plot: In the 1980s in Boston, Irish Mob boss Frank Costello introduces himself to a young Colin Sullivan.\nMany years later, Sullivan has been groomed as a spy inside the Massachusetts State Police (MSP) and joins the Special Investigation Unit (SIU). Another police academy recruit, Billy Costigan, is selected by Captain Queenan and Sergeant Dignam to go undercover as a criminal.\nCostigan serves a term in prison to set up his cover and further commits a series of crimes. Sullivan begins dating Madolyn Madden, a psychiatrist. Costigan catches the attention of Costello, who later recruits Costigan into his organization. Over the next year, Costigan becomes increasingly involved in the organization. His mental state declines but Queenan and Dignam convince him to continue. Costigan begins having appointments with Madolyn.\nThe MSP and Costello realize there is a mole in either o

2024-08-15 23:40:47.477264: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 23:40:47.477321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 23:40:47.478762: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

Schindler's List [Document(metadata={'source': 'The Departed'}, page_content="Movie name: The Departed\n\n\n\nIts plot: In the 1980s in Boston, Irish Mob boss Frank Costello introduces himself to a young Colin Sullivan.\nMany years later, Sullivan has been groomed as a spy inside the Massachusetts State Police (MSP) and joins the Special Investigation Unit (SIU). Another police academy recruit, Billy Costigan, is selected by Captain Queenan and Sergeant Dignam to go undercover as a criminal.\nCostigan serves a term in prison to set up his cover and further commits a series of crimes. Sullivan begins dating Madolyn Madden, a psychiatrist. Costigan catches the attention of Costello, who later recruits Costigan into his organization. Over the next year, Costigan becomes increasingly involved in the organization. His mental state declines but Queenan and Dignam convince him to continue. Costigan begins having appointments with Madolyn.\nThe MSP and Costello realize there is a mole in eithe

2024-08-15 23:41:11.758384: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 23:41:11.758438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 23:41:11.759863: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


The Lord of the Rings: The Return of the King [Document(metadata={'source': 'The Lord of the Rings: The Fellowship of the Ring'}, page_content='Movie name: The Lord of the Rings: The Fellowship of the Ring\n\n\n\nIts plot: In the Second Age of Middle-earth, the lords of Elves, Dwarves, and Men are given Rings of Power. Unbeknownst to them, the Dark Lord Sauron forges the One Ring in Mount Doom, instilling into it a great part of his power to dominate the other Rings and conquer Middle-earth. A final alliance of Men and Elves battles Sauron\'s forces in Mordor. Isildur of Gondor severs Sauron\'s finger and the Ring with it, thereby vanquishing Sauron and returning him to spirit form. With Sauron\'s first defeat, the Third Age of Middle-earth begins. The Ring\'s influence corrupts Isildur, who takes it for himself and is later killed by Orcs. The Ring is lost in a river for 2,500 years until it is found by Gollum, who owns it for over four and a half centuries. The Ring abandons Gollum a

2024-08-15 23:41:23.824301: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 23:41:23.824365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 23:41:23.825813: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/conda/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/conda/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/opt/conda/lib/python3.10/site

KeyboardInterrupt: 